# Data Everywhere

In [1]:
import pandas as pd 
import requests

## BLS Data - CPI

In [4]:
url = "https://api.bls.gov/publicAPI/v2/timeseries/data/"
payload = {"seriesid":["CUUR0000SA0","LNS14000000"], "startyear":"2018", "endyear":"2025"}
data = requests.post(url, json=payload).json()["Results"]["series"]

rows = []
for s in data:
    sid = s["seriesID"]
    for o in s["data"]:
        rows.append({"series":sid,"year":int(o["year"]),"period":o["period"],"value":float(o["value"])})
df = pd.DataFrame(rows)

In [5]:
df.head()

,series,year,period,value
0,CUUR0000SA0,2025,M09,324.800
1,CUUR0000SA0,2025,M08,323.976
2,CUUR0000SA0,2025,M07,323.048
3,CUUR0000SA0,2025,M06,322.561
4,CUUR0000SA0,2025,M05,321.465


## US Census
- MDAT (Microdata Access Tool): build custom tables from anonymized PUMS (ACS/CPS).
- API: query by dataset + variables + geography.

In [6]:
base = "https://api.census.gov/data/2023/acs/acs1/subject"
params = {
    "get":"NAME,S0101_C01_001E",  # total population
    "for":"state:*"
}
res = requests.get(base, params=params).json()
census = pd.DataFrame(res)
#census = pd.DataFrame(res[1:], columns=res[0])

In [7]:
census.head()

,0,1,2
0,NAME,S0101_C01_001E,state
1,Alabama,5108468,01
2,Alaska,733406,02
3,Arizona,7431344,04
4,Arkansas,3067732,05


## CDC

In [17]:
url = "https://data.cdc.gov/resource/swc5-untb.json"  # PLACES: County, 2024 release
params_us = {
    "$select": "year,stateabbr,locationname,countyfips,measureid,datavaluetypeid,data_value",
    "$where": 'year="2022" AND measureid="CASTHMA" AND datavaluetypeid="CrdPrv"',
    "$order": "data_value DESC",
    "$limit": 10
}
headers = {}
r = requests.get(url, params=params, headers=headers, timeout=30)
r.raise_for_status()
data = r.json()

df = pd.DataFrame(data)

In [18]:
df.head()

,year,stateabbr,statedesc,locationname,measureid,datavaluetypeid,data_value
0,2022,OH,Ohio,Pike,CASTHMA,CrdPrv,12.4
1,2022,OH,Ohio,Holmes,CASTHMA,CrdPrv,12.3
2,2022,OH,Ohio,Vinton,CASTHMA,CrdPrv,12.1
3,2022,OH,Ohio,Lawrence,CASTHMA,CrdPrv,12.0
4,2022,OH,Ohio,Trumbull,CASTHMA,CrdPrv,12.0
